# Notebook 8 - Représentation des connaissances

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

CSI4506 Intelligence Artificielle  
Automne 2021 \
Versions 1 (2020) préparée par Julian Templeton, Caroline Barrière et Joel Muteba.  Version 2 (2021) modifiée par Caroline Barrière.

***INTRODUCTION***:  

Lors de la lecture de texte, comprendre les type d'entités utilisées dans le texte permet d'inférer des informations supplémentaires sur ces entités.  Par exemple, si un texte mentionne *Canada*, le fait de savoir que c'est une GPE (entité géo-politique), nous indique déjà que cette entité a une supercifie, une population, etc.  Grâce à l'utilisation de la reconnaissance d'entités nommées (Named Entity Recognition, NER), nous sommes en mesure de déterminer si une entité est une personne, une organisation, un pays, ... 

Lors de l'exploration de texte en version électronique, nous voyons aussi occasionnellement que les entités ont des liens cliquables vers des pages Web avec plus d'informations sur l'entité. Il s'agit d'une forme d'amélioration du texte pour permettre aux lecteurs d'accéder facilement à des informations supplémentaires.  Si nous prenons encore l'exemple de *Canada*, si nous le transformons en [Canada](https://en.wikipedia.org/wiki/Canada), à l'aide du linking d'entités (entity linking) nous accédons à d'avantage d'informations. 

Dans ce notebook, nous revisiterons l'ensemble de données d'actualités liées à Covid-19 du notebook 7 pour explorer comment nous pouvons améliorer les résultats de NER de spaCy et aider à la compréhension des articles de presse grâce à l'utilisation du linking d'entités. Cela se fera en trois parties, soit 

(1) nous explorerons d'abord les résultats du NER de spaCy \
(2) nous utiliserons la cohérence du texte pour un post-traitement au NER de spaCy\
(3) puis nous effectuons des ajouts (*enrichissement*) au texte avec le linking d'entités.

Ce notebook utilise des bibliothèques qui ont été utilisées dans les notebooks précédents, notamment spaCy et pandas. 

***DEVOIR***:  

Parcourez le notebook en exécutant chaque cellule, une à la fois. \
Recherchez **(TO DO)** pour les tâches que vous devez effectuer. Ne modifiez pas le code en dehors des questions auxquelles vous êtes invité à répondre à moins que cela ne vous soit spécifiquement demandé. Une fois que vous avez terminé, signez le notebook (à la fin du notebook), renommez-le *NumEtudiant-NomFamille-Notebook8.ipynb* et soumettez-le.

*Le notebook sera noté le 30.  \
Chaque **(TO DO)** est associé à un certain nombre de points.*
***

In [1]:
# Before starting we will import every module that we will be using
import spacy
import pandas as pd

In [5]:
# The core spacy object can be used for tokenization, lemmatization, POS Tagging, NER ...
# Note that this is specifically for the English language and requires the English package to be installed
# via pip to work as intended.

# sp = spacy.load('en')

# If the above causes an error then install the package as below
# !spacy download en_core_web_sm
sp = spacy.load('en_core_web_sm')

Comme pour le dernier notebook, l'ensemble de données est fourni sur Brightspace (Module 8) avec ce notebook, mais les détails le concernant peuvent être trouvés [ici](https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26?select=news.csv). La première chose que nous allons faire, comme d'habitude, est de charger le fichier dans un dataframe pandas.

In [6]:
# Read the dataset, show top ten rows
df = pd.read_csv("news.csv")
df.head(10)

,Unnamed: 0,authors,title,publish_date,description,text,url
0,0,[],'More vital now:' Gay-straight alliances go vi...,2020-05-03 1:30,Lily Overacker and Laurell Pallot start each g...,Lily Overacker and Laurell Pallot start each g...,https://www.cbc.ca/news/canada/calgary/gay-str...
1,1,[],Scientists aim to 'see' invisible transmission...,2020-05-02 8:00,Some researchers aim to learn more about how t...,"This is an excerpt from Second Opinion, a week...",https://www.cbc.ca/news/technology/droplet-tra...
2,2,['The Canadian Press'],Coronavirus: What's happening in Canada and ar...,2020-05-02 11:28,Canada's chief public health officer struck an...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
3,3,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
4,4,[],"B.C. announces 26 new coronavirus cases, new c...",2020-05-02 18:45,B.C. provincial health officer Dr. Bonnie Henr...,B.C. provincial health officer Dr. Bonnie Henr...,https://www.cbc.ca/news/canada/british-columbi...
5,5,"['Senior Writer', 'Chris Arsenault Is A Senior...",Brazil has the most confirmed COVID-19 cases i...,2020-05-02 8:00,"From describing coronavirus as a ""little flu,""...","With infection rates spiralling, some big city...",https://www.cbc.ca/news/world/brazil-has-the-m...
6,6,['Cbc News'],The latest on the coronavirus outbreak for May 1,2020-05-01 20:43,The latest on the coronavirus outbreak from CB...,Coronavirus Brief (CBC) Canada is officiall...,https://www.cbc.ca/news/the-latest-on-the-coro...
7,7,['Cbc News'],Coronavirus: What's happening in Canada and ar...,2020-05-01 11:51,Nova Scotia announced Friday it is immediately...,The latest: The lives behind the numbers: Wha...,https://www.cbc.ca/news/canada/coronavirus-cov...
8,8,"['Senior Writer', ""Adam Miller Is Senior Digit...",Did the WHO mishandle the global coronavirus p...,2020-04-30 8:00,The World Health Organization has come under f...,The World Health Organization has come under f...,https://www.cbc.ca/news/health/coronavirus-who...
9,9,['Thomson Reuters'],Armed people in Michigan's legislature protest...,2020-04-30 21:37,"Hundreds of protesters, some armed, gathered a...","Hundreds of protesters, some armed, gathered a...",https://www.cbc.ca/news/world/protesters-michi...


# New section

**PARTIE 1 - Le NER de spaCy**  
  
Commençons par regarder le NER qui est effectué par spaCy. La documentation de SpaCy ne nous dit pas exactement comment leur NER est fait (certainement leur secret commercial), mais nous pouvons au moins regarder les résultats.

Comme nous en avons discuté dans les notebooks précédents, lors de l'évaluation d'un processus, d'un modèle ou d'un outil, nous pouvons faire une évaluation quantitative ou une **évaluation qualitative** des résultats. Dans ce notebook, nous travaillons à un niveau qualitatif, ce qui signifie que nous ne mesurons pas des métriques telles que la précision/rappel, mais imprimons plutôt les résultats de quelques exemples et essayons de comprendre ces résultats.


Vous trouverez ci-dessous la même phrase exemple que dans le dernier Notebook, pour laquelle nous avions examiné l'étiquetage des parties du discours (POS tagging) et d'autres processus linguistiques.  Nous utilisons cette phrase exemple pour illustrer maintenant comment obtenir les prédictions de type NER de spaCy pour les tokens dans un texte.

In [7]:
# Same example from notebook 7, recall that we loop through the iterator found in the .ents property of a parsed sentence
sentence_example = "Government guidelines in Canada recommend that people stay at least two metres away from others as part of physical distancing measures to curb the spread of COVID-19."
sentence_example_content = sp(sentence_example)
# Loop through all tokens that contain a NER type and print the token along with the corresponding NER type
for token in sentence_example_content.ents:
    print("\"" + token.text + "\" is a " + token.label_ )

"Canada" is a GPE
"at least two metres" is a QUANTITY


**(TO DO) Q1 - 5 points**  

Dans le texte du ***second document*** (index 1) de notre corpus de documents, quels mots sont *PER* (spaCy utilise le type *PERSON*, plutôt que *PER*), *ORG* (Organisation) et *GPE* (entité géopolitique). Vous devez effectuer les opérations suivantes pour cette question:

a) (2 points) Imprimez chaque *PERSON*, *ORG* et *GPE* avec son type NER tel que trouvé par spaCy.

b) (1 points) Est-ce que la majorité des prédictions de spaCy sont correctes? Donnez deux exemples de sorties obtenues en (a) qui sont incorrectes selon vous.

c) (2 points) Il arrive parfois que des problèmes avec les prédictions de type NER proviennent d'erreurs dans des étapes antérieures dans la pipeline TAL (e.g. tokenization, POS tagging).  Utilisez 2 exemples de sorties obtenues en (a) pour illustrer cette possibilité, et tentez d'offrir un diagnostic (que s'est-il passé?).

In [8]:
# RÉPONSE Q1(a) - 2 points
# Select the second document (index 1)
doc = df["text"][1]

doc_sp = sp(doc)

# SHOW PERSON, ORG, GPE
for i, token in enumerate(doc_sp.ents):
    if(token.label_=="PERSON" or token.label_=="GPE" or token.label_=="ORG"  ):
        print(str(i) + ": \"" + token.text + "\" is a " + token.label_ )

3: "the World Health Organization" is a ORG
4: "Touches" is a ORG
5: "WHO" is a ORG
7: "the Public Health Agency" is a ORG
8: "Canada" is a GPE
14: "W.F. Wells" is a PERSON
15: "Harvard School of Public Health" is a ORG
17: "Wells" is a PERSON
18: "INTERACTIVECoronavirus" is a ORG
20: "Canada" is a GPE
22: "Lydia Bourouiba" is a PERSON
23: "the Fluid Dynamics of Disease Transmission Laboratory" is a ORG
24: "the Massachusetts Institute of Technology" is a ORG
25: "Bourouiba" is a PERSON
27: "Mark Loeb" is a PERSON
28: "Hamilton" is a PERSON
29: "McMaster University" is a ORG
30: "RNA" is a ORG
31: "Wuhan" is a GPE
32: "China" is a GPE
33: "Nebraska" is a GPE
34: "Loeb" is a PERSON
35: "Loeb" is a PERSON
36: "Canada" is a GPE
38: "Gary Moore/CBC" is a PERSON
39: "Allison McGeer" is a PERSON
40: "Sinai Health" is a ORG
41: "Toronto" is a GPE
44: "McGeer" is a ORG
46: "McGeer" is a PERSON
48: "Bourouiba's" is a ORG
49: "Bourouiba" is a ORG
51: "Credit Lydia Bourouiba/MIT" is a ORG
56: "Sa

**RÉPONSE Q1(b) - 1 point**   

La majorite oui mais pas tout

exemples incorrects:

"Bourouiba" is a ORG

"McGeer" is a ORG


**RÉPONSE Q1 (c) - 2 points**   
Exemple:

44: "McGeer" is a ORG
49: "Bourouiba" is a ORG

L'algorithme a surement fait une erreur de POS tagging du a la position de l'entité, et donc un type NER different de celui qui le qualifie

**PARTIE 2 - Cohérence du texte et chaînes de coréférences**  

Comme vous avez vu, les résultats du spaCy NER sont très bons, mais pas parfait.  Un problème principal avec NER (pas seulement dans spaCy mais dans de nombreux outils) est que l'annotation est effectuée une entité à la fois sans tenir compte du document global.

Mais en regardant l'ensemble du document, et sachant que le texte est généralement cohérent, nous pouvons effectuer un post-traitement dans le module NER de spaCy et corriger certaines erreurs. Par texte cohérent, nous entendons, par exemple, que si une personne est désignée avec un nom particulier, par ex. *McGeer*, il y a de fortes chances qu'à chaque fois que l'on voit *McGeer* dans le document, ce soit la même personne.  Toutes les mentions *McGeer* formeraient une chaîne de coréférences vers la même entité.  Il est donc peu probable que *McGeer* soit une fois une personne et une fois une organisation. Ce n'est pas toujours vrai, il existe de nombreux contre-exemples, mais c'est une hypothèse courante. Cette idée est même le sujet d'un article de la PNL plus ancien et très cité intitulé « One sense per discourse » (Gale et al. 1992).

Avec cette idée de "One sense per discourse", nous explorerons deux stratégies différentes pour utiliser la cohérence du texte pour post-traiter la sortie du module spaCy NER.

La première stratégie (*explorée en Q2 / Q3*) est de trouver, parmi tous les types de NER assignés, lequel est le plus fréquent. Par exemple, l'entité *Bourouiba* s'est vu attribuer 1 fois ORG et 2 fois PERSON, donc ces informations peuvent être utilisées pour modifier le type ORG et le changer en PERSON.

La deuxième stratégie (explorée à la Q4) est d'essayer de trouver une forme plus longue dans le texte. Puisque cette forme plus longue devrait être moins ambiguë, nous pouvons l'utiliser pour lever l'ambiguïté des formes plus courtes et plus ambiguës. Par exemple, *Lydia Bourouiba* apparaît dans le texte et se voit attribuer PERSON. Nous pouvons utiliser cette information pour attribuer à d'autres occurrences de la forme abrégée *Bourouiba* le même type PERSON.

Bien sûr, utiliser ces méthodes pour la cohérence du texte ne fonctionnera pas à tous les coups, et introduira malheureusement quelques erreurs... Mais essayons. C'est le but des études empiriques, nous essayons des idées.

Reprenons notre nouvelle utilisée pour Q1, mais cette fois, montrons non seulement GPE, PER, ORG, mais plutôt toutes les entités nommées trouvées par spaCy.

In [9]:
# Select document 2
doc = df["text"][1]
# NER
doc_sp = sp(doc)

# Display all entities from the text along with their index in the .ents iterator and the
# corresponding NER type
for i, token in enumerate(doc_sp.ents):
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ )

0: "Saturday" is a DATE
1: "morning" is a TIME
2: "two metres" is a QUANTITY
3: "the World Health Organization" is a ORG
4: "Touches" is a ORG
5: "WHO" is a ORG
6: "more than one metre" is a QUANTITY
7: "the Public Health Agency" is a ORG
8: "Canada" is a GPE
9: "at least two metres" is a QUANTITY
10: "two" is a CARDINAL
11: "2 metres" is a QUANTITY
12: "the 19th century" is a DATE
13: "1934" is a DATE
14: "W.F. Wells" is a PERSON
15: "Harvard School of Public Health" is a ORG
16: "two metres" is a QUANTITY
17: "Wells" is a PERSON
18: "INTERACTIVECoronavirus" is a ORG
19: "56,000" is a CARDINAL
20: "Canada" is a GPE
21: "Saturday" is a DATE
22: "Lydia Bourouiba" is a PERSON
23: "the Fluid Dynamics of Disease Transmission Laboratory" is a ORG
24: "the Massachusetts Institute of Technology" is a ORG
25: "Bourouiba" is a PERSON
26: "Canadian" is a NORP
27: "Mark Loeb" is a PERSON
28: "Hamilton" is a PERSON
29: "McMaster University" is a ORG
30: "RNA" is a ORG
31: "Wuhan" is a GPE
32: "Chi

**(TO DO) Q2 - 3 points**  
Comme vous pouvez le voir dans les résultats, parfois la même entité s'est vu attribuer différents types d'entités.  Par exemple, *McGeer* est une fois ORG, une fois PERSON, puisque l'algorithme NER regarde phrase par phrase. Dans la fonction suivante, le but sera de trouver tous les types d'entités possibles affectés à une seule entité.

Complétez la définition de la fonction *find_entity_types* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* et une liste de toutes les entités spaCy du définies par le paramètre *entities*.

La fonction doit trouver toutes les entités ayant la même forme de surface que *entity* dans l'ensemble *entities*. Pour chaque correspondance entre les entités, ajoutez le type NER trouvé au dictionnaire *type_counts* et mettez à jour la fréquence de ce type.

Le dictionnaire *type_counts* contiendrait par exemple *McGeer* avec ORG = 1, et PERSON = 1, car la fonction a trouvé 2 mentions de *McGeer*, chacune avec un type différent.

In [27]:
# RÉPONSE Q2
import collections
def find_entity_types(entity, entities):
    '''
    Given a specific entity and a list of entities, finds all entities from the list that match surface form of the specified
    entity, but that could be of a different type.
    
    Returns the different NER types that have been classified for an entity and the count per NER type
    as a dictionary with the keys as the NER type and the value as the count
    '''
    
    type_counts = {}
    for tok in entities:
        if(tok.text==entity.text):
            if tok.label_  not in type_counts:
               
                type_counts[tok.label_] = 1
                
            else:
               
                type_counts[tok.label_] += 1
               
    return type_counts


In [46]:
# Test the above to find the result when checking for the types of the entity 'Bourouiba' 
# from the document loaded above
print("All possible NER types for \"" + doc_sp.ents[49].text + "\" are " + str(find_entity_types(doc_sp.ents[49], doc_sp.ents)))

All possible NER types for "Bourouiba" are {'PERSON': 2, 'ORG': 1}


**(TO DO) Q3 - 2 points**  

Dans la méthode précédente, *find_entity_types*, nous avons trouvé tous les types d'entités possibles pour chaque mention. Par exemple, dans le cas de *McGeer*, c'est une égalité. Mais pour *Bourouiba*, il existe un type ORG et 2 types PERSON, donc le plus courant serait PERSON.

Complétez la définition de la fonction *most_common_type* ci-dessous. Cette fonction accepte en entrée une entité spaCy spécifique définie par le paramètre *entity* et une liste de toutes les entités spaCy définies par le paramètre *entities*.

Remarque: vous pouvez régler les cas d'égalité à votre guise.  Aussi, assurez-vous d'utiliser la méthode *find_entity_types* que vous avez écrite précédemment.


In [41]:
# RÉPONSE Q3 

def most_common_type(entity, entities):
    '''
    Given a specific entity and a list of entities, find the most similar entities and assign the
    NER type to entity based on the most common NER type assigned to entities of the same name (if there
    is a tie, you decide how to handle this).
    
    Returns the most common NER type based on similar entities
    '''
    # TODO
 
    a=""
    for tok in entities:
        if(tok.text==entity.text):
            find_entity_types(tok,entities)
            if(tok.label_>entity.label_):
              a = (tok.label_)
            else:
              a =(entity.label_)
            
    return a 


In [48]:
# Test the above to find the result when checking for the types of the entity 'Bourouiba' 
# from the document loaded above
print("The most common NER type to \"" + doc_sp.ents[49].text + "\" is " + str(most_common_type(doc_sp.ents[49], doc_sp.ents)))

The most common NER type to "Bourouiba" is PERSON



Notre première exploration (au Q2/Q3) portait sur la fréquence d'occurrence. Nous avons supposé que le type d'entité le plus courant pourrait être le bon. Maintenant, nous allons explorer l'idée que la mention la moins ambiguë à une entité pourrait être la bonne. Par exemple, *McGeer* est plus ambigu (forme plus courte) que *Allison McGeer* (forme plus longue). Souvent, la forme la plus longue de référence à une entité est la moins ambiguë. Mais parce que cette forme est longue à écrire, nous l'utilisons souvent avec parcimonie dans un texte (peut-être une seule fois) et les mentions subséquentes de la même entité utiliseront la forme courte. Par exemple, le texte peut mentionner *Allison McGeer* une fois, puis utiliser la forme abrégée *McGeer* pour faire référence à la même personne plusieurs fois dans le document.

Dans les vidéos du cours, nous avons parlé de chaîne de coréférences.  Ainsi, une chaîne contient des mentions longues et courtes, référant toutes à la même entités.

La forme plus longue est souvent appelée forme normalisée, et c'est une forme que nous sommes susceptibles de trouver dans une ressource externe. Nous verrons dans la partie 3 de ce Notebook, lorsque nous ferons des liens d'entités, qu'il existe une entrée Wikipedia pour *Allison McGeer* vers laquelle nous pourrions établir un lien. Nous pouvons considérer la forme plus longue de *Allison McGeer* comme la forme normalisée.

**(TO DO) Q4 (a) - 3 points**  

Vous devez écrire une fonction qui trouvera la forme la plus longue pouvant correspondre à une mention.

Votre fonction aura les mêmes paramètres *entity* et *entities*, mais cette fois la fonction devra attribuer à *entité* le type NER d'une autre entité dans l'itérateur *entities*, soit le NER de la forme la plus longue.

Plus précisément, vous devez parcourir les *entités* pour trouver une forme normalisée de *entité*. Dans ce scénario, l'entité avec la forme la plus longue contenant *entité* en tant que sous-chaîne sera considérée comme la forme normalisée et sera retournée.

Ex : *Lydia Bourouiba* est la forme normalisée de *Bourouiba*. Ainsi, l'entité ayant cette forme doit être retournée. Mais *McMaster University* est déjà la forme la plus longue, donc si nous recherchons une forme normalisée pour cette entité, la fonction devrait renvoyer l'entité elle-même.

In [51]:
# RÉPONSE Q4(a)
# Find the longest surface form within "entities" for which the surface for of "entity" is a substring

def assign_normalized_form(entity, entities):
    a=entity
    for substring in entities:
        if (entity.text in substring.text): 
            if (len(entity.text)<len(substring.text)):
                entity = substring
                a = substring
                
        else:
            a=entity
        
    return a
        
            
assign_normalized_form(doc_sp.ents[44], doc_sp.ents)

Allison McGeer

Testons la fonction ci-haut, en supposant que les candidats se retrouvent uniquement dans les mentions précédentes, car souvent une forme longue est d'abord donnée *Allison McGeer* et les formes subséquentes sont les formes courtes *McGeer*.

In [52]:
# Testing using only the previous references as candidates
test = df["text"][1]
# Parse the text with spaCy
test_sp = sp(test)
for i, token in enumerate(test_sp.ents):
    ent = assign_normalized_form(test_sp.ents[i], test_sp.ents[0:i-1])
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ + "  " + ent.text + "  " + ent.label_)

0: "Saturday" is a DATE  Saturday  DATE
1: "morning" is a TIME  morning  TIME
2: "two metres" is a QUANTITY  two metres  QUANTITY
3: "the World Health Organization" is a ORG  the World Health Organization  ORG
4: "Touches" is a ORG  Touches  ORG
5: "WHO" is a ORG  WHO  ORG
6: "more than one metre" is a QUANTITY  more than one metre  QUANTITY
7: "the Public Health Agency" is a ORG  the Public Health Agency  ORG
8: "Canada" is a GPE  Canada  GPE
9: "at least two metres" is a QUANTITY  at least two metres  QUANTITY
10: "two" is a CARDINAL  two metres  QUANTITY
11: "2 metres" is a QUANTITY  2 metres  QUANTITY
12: "the 19th century" is a DATE  the 19th century  DATE
13: "1934" is a DATE  1934  DATE
14: "W.F. Wells" is a PERSON  W.F. Wells  PERSON
15: "Harvard School of Public Health" is a ORG  Harvard School of Public Health  ORG
16: "two metres" is a QUANTITY  at least two metres  QUANTITY
17: "Wells" is a PERSON  W.F. Wells  PERSON
18: "INTERACTIVECoronavirus" is a ORG  INTERACTIVECoronav

**(TO DO) Q4(b) - 2 points**

Faites d'autres tests sans vous limiter à chercher des formes de surfaces plus longues mentionnées avant une entité (voir *test_sp.ents[0:i-1]* dans le code ci-haut), et chercher avant ou après. Ou chercher dans un intervalle (par exemple, max N entités avant ou après). Est-ce que cela fait une différence? Expliquez ce que vous avez testé et fournissez au moins 2 exemples de changements que vous remarquez.

In [53]:
# RÉPONSE Q4(b)
# Do a different test
test = df["text"][1]
# Parse the text with spaCy
test_sp = sp(test)
for i, token in enumerate(test_sp.ents):
    ent = assign_normalized_form(test_sp.ents[i], test_sp.ents[:-2]) #everything except the last two items
    print(str(i) + ": \"" + token.text + "\" is a " + token.label_ + "  " + ent.text + "  " + ent.label_)

0: "Saturday" is a DATE  Saturday  DATE
1: "morning" is a TIME  morning  TIME
2: "two metres" is a QUANTITY  at least two metres  QUANTITY
3: "the World Health Organization" is a ORG  the World Health Organization  ORG
4: "Touches" is a ORG  Touches  ORG
5: "WHO" is a ORG  WHO  ORG
6: "more than one metre" is a QUANTITY  more than one metre  QUANTITY
7: "the Public Health Agency" is a ORG  the Public Health Agency  ORG
8: "Canada" is a GPE  Canada  GPE
9: "at least two metres" is a QUANTITY  at least two metres  QUANTITY
10: "two" is a CARDINAL  at least two metres  QUANTITY
11: "2 metres" is a QUANTITY  2 metres  QUANTITY
12: "the 19th century" is a DATE  the 19th century  DATE
13: "1934" is a DATE  1934  DATE
14: "W.F. Wells" is a PERSON  W.F. Wells  PERSON
15: "Harvard School of Public Health" is a ORG  Harvard School of Public Health  ORG
16: "two metres" is a QUANTITY  at least two metres  QUANTITY
17: "Wells" is a PERSON  W.F. Wells  PERSON
18: "INTERACTIVECoronavirus" is a ORG  

**RÉPONSE Q4(b)**

On remarque quelques differences oui.

Dans le deuxieme test, on a essaye de compter en commencant par la fin pour aller au commencement

Un exemple est 2- "two metres" qui dans le premier test renvoie lui meme mais renvoie sa forme la plus longue "at least two metres" dans le deuxieme test

Un aautre exemple est 10: "two" qui dans le premier test renvoie "two metres" mais renvoie sa forme la plus longue "at least two metres " dans le deuxieme test

**(TO DO) Q5 - 5 points**  

Utilisez un autre article de nouvelle dans le corpus, le 7e article, donc index 6.

(a) (2 points) Exécutez les deux approches (NER la plus fréquente, NER de la forme la plus longue). Pour chaque entité trouvée dans le texte, imprimez son type d'entité d'origine (tel que trouvé par spaCy, puis le type d'entité le plus courant (résultat de Q3), puis la forme normalisée avec son type d'entité (résultat de Q4). \
(b) (3 points) Analyser et discuter les résultats. Pensez-vous que ces approches de cohérence de texte aident ou sont-elles trop simples ? Y a-t-il des résultats contradictoires (les deux approches donnent des résultats différents). Si oui, montrez des exemples différents.



In [57]:
# RÉPONSE Q5(a) 
# Select document index 6
doc = df["text"][6]

docs =sp(doc)

# Display all entities from the text along with their index in the .ents iterator and the
# corresponding NER type
for i, token in enumerate(docs.ents):
    freq = most_common_type(docs.ents[i], docs.ents)
    ent = assign_normalized_form(docs.ents[i], docs.ents[0:i-1])
    print(str(i) + ": "  + token.text + " is a " + token.label_+ "... Forme la plus longue:  " + ent.text + " is a " + ent.label_ + "... Type plus fréquent "+ str(freq ))
    

0: Coronavirus Brief is a ORG... Forme la plus longue:  Coronavirus Brief is a ORG... Type plus fréquent ORG
1: CBC is a ORG... Forme la plus longue:  CBC is a ORG... Type plus fréquent ORG
2: Canada is a GPE... Forme la plus longue:  Canada is a GPE... Type plus fréquent GPE
3: C.D. Howe is a PERSON... Forme la plus longue:  C.D. Howe is a PERSON... Type plus fréquent PERSON
4: Monday is a DATE... Forme la plus longue:  Monday is a DATE... Type plus fréquent DATE
5: Alberta is a GPE... Forme la plus longue:  Alberta is a GPE... Type plus fréquent GPE
6: first is a ORDINAL... Forme la plus longue:  first is a ORDINAL... Type plus fréquent ORDINAL
7: Saturday is a DATE... Forme la plus longue:  Saturday is a DATE... Type plus fréquent DATE
8: Air Canada is a ORG... Forme la plus longue:  Air Canada is a ORG... Type plus fréquent ORG
9: Christmas is a DATE... Forme la plus longue:  Christmas is a DATE... Type plus fréquent DATE
10: Canadians is a NORP... Forme la plus longue:  Canadians 

**RÉPONSE Q5(b) - 3 point**     

Cette façon de faire, qui présente certes des avantages, n'est guère parfaite, notamment parce qu’elle n’est pas structurante, au sens où elle n’établit pas de liens explicites avec les règles qui définissent ce qu’est un bon texte en rapport avec sa coherence. 

Oui, On peut noter quelques resultats qui sont en fait contradictoires dans les deux approches 

Par exemple: 

17: Canada is a GPE... Forme la plus longue:  Air Canada is a ORG... Type plus fréquent GPE

38: Calgary is a GPE... Forme la plus longue:  the University of Calgary is a ORG... Type plus fréquent GPE

more than is a CARDINAL... Forme la plus longue:  more than $1.2 million is a MONEY... Type plus fréquent CARDINAL


**PARTIE 3 - Linking d'entité / Enrichissement du texte**  

Pour la troisième partie de ce notebook, nous explorerons comment nous pouvons fournir une "valeur ajoutée" au texte des documents. Dans ce scénario, nous enrichirons le texte en effectuant un linking d'entité. 

Cela signifie que nous tenterons de relier les entités détectées par le NER de spaCy à une page Web active sur laquelle un lecteur peut cliquer pour obtenir plus d'informations concernant l'entité. Wikipedia est une très bonne ressource pour trouver plus d'informations sur une entité, et nous utiliserons cette ressource pour le linking d'entités.

Avant de tenter de faire cette opération automatiquement, voici un exemple de la façon dont un texte sans linking d'entité se compare à un texte enrichi avec linking d'entité fait manuellement:

*Aucun linking d'entité:* \
Pendant la pandémie, des villes américaines telles que Atlanta, Chicago et Denver ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

*Avec linking d'entité:* \
Pendant la pandémie, des villes américaines telles que <a href="http://en.wikipedia.org/wiki/Atlanta"> Atlanta </a>, <a href = "http://en.wikipedia.org/wiki / Chicago "> Chicago </a> et <a href="http://en.wikipedia.org/wiki/Denver"> Denver </a> ont apporté plusieurs ajustements à leurs systèmes de transport en commun.

Transformer un texte automatiquement avec des liens cliquables demande plusieurs traitements au niveau des chaînes de caractères.  Nous nous contenterons dans ce Notebook de trouver les liens sans faire les remplacements directement dans le texte.  Cela nous permettra d'explorer la ressource Wikipedia, et comprendre les difficultés relatives au "entity linking" sans perdre trop de temps dans la manipulation complexe de chaînes de caractères.

Par exemple, avec le document (index 6), nous voudrions pouvoir lier les entités trouvées par spaCy à la page wikipedia la plus probable donnant accès à de l'information supplémentaire sur cette entité.

**Le format ci-bas de liste enrichie est le type de résultat demandé à la question Q6 ci-après.**  Pour simplifier votre code, nous utiliserons ce genre de sortie plutôt que d'avoir les liens directement dans le texte.

0: "Coronavirus Brief" is a ORG found at http://en.wikipedia.org/wiki/Coronavirus_Brief \
1: "CBC" is a ORG found at http://en.wikipedia.org/wiki/CBC \
2: "Canada" is a GPE found at http://en.wikipedia.org/wiki/Canada \
3: "C.D. Howe" is a PERSON found at http://en.wikipedia.org/wiki/C.D._Howe \
4: ... \



**(TO DO) Q6 - 5 points**  \
Écrivez le code nécessaire pour faire la recherche d'une page wikipedia pour les entités trouvées par spaCy (tel qu'illustré ci-haut) dans un document particulier.

*Vous pouvez écrire ce code comme vous le souhaitez, mais il faut inclure les éléments suivants :*

* (a) Une restriction sur le type d'entités que vous liez. Par exemple, Wikipedia ne contient pas de quantités (tel "two meters") il serait alors inadéquat d'inclure un lien vers une quantité.
* (b) L'utilisation de la *forme normalisée* de l'entité pour effectuer la liaison. Par exemple, *Allison McGeer* a une page Wikipédia (https://en.wikipedia.org/wiki/Allison_McGeer) vers laquelle vous pouvez créer un lien, même lorsque vous regardez l'entité avec l'étiquette *McGeer*. Assurez-vous donc d'utiliser la fonction que vous avez développée à la Question 4 (Q4).
* (c) Attention : la page wikipedia utilise des traits de soulignement. Ainsi, par exemple, *McMaster University* doit être transformé en https://en.wikipedia.org/wiki/McMaster_University (avec un trait de soulignement entre *McMaster* et *University*)
* (d) Inclure un élément de post-traitement sur la forme de surface la plus longue trouvée. Par exemple, *the C.D. Howe Institute's* est trouvé par spaCy, mais Wikipédia contiendra *C.D._Howe_Institute*. Vous pouvez supprimer les petites particules comme *the* pour augmenter les chances de lien.
* (e) Pour un document à l'entrée, imprimez une liste des formes de surface, type d'entité et lien vers Wikipedia (tel que montré ci-haut)

Assurez-vous de mettre des commentaires dans votre code pour indiquer clairement ce qui correspond aux parties (a), (b), (c), (d) et (e).

Il y aura probablement de nombreux liens qui mènent vers des pages Wikipedia inexistante.  C'est bon, ne vous inquiétez pas pour ça. Wikipédia ne contient pas tout, et certaines formes normalisées n'y seront pas. On vous demandera de discuter les résultats du linking à la question 7.


In [58]:
# RÉPONSE Q6 -

doc = df["text"][6]

docss =sp(doc)

for i, token in enumerate(docss.ents):
    if(token.label_ =="QUANTITY") or (token.label_=="MONEY"): #a- Une restriction sur le type d'entités que vous liez.
        print("")
    else:
        ent = assign_normalized_form(docss.ents[i], docss.ents[0:i-1]) # b-  Assurez-vous donc d'utiliser la fonction que vous avez développée à la Question 4 (Q4).
    
        tok = '_'.join(ent.text.split())        # c- la page wikipedia utilise des traits de soulignement. Ainsi, par exemple, McMaster University doit être transformé en 
    
        delpar = tok.replace("'s", "")        # d- inclure un élément de post-traitement sur la forme de surface la plus longue trouvée. Par exemple, the C.D. Howe Institute's est trouvé par spaCy, mais Wikipédia contiendra C.D._Howe_Institute.

        print(str(i) + ": \"" +token.text +"\" is a "+ token.label_ , "found at https://en.wikipedia.org/wiki/"+ delpar)   #e- Pour un document à l'entrée, imprimez une liste des formes de surface, type d'entité et lien vers Wikipedia (tel que montré ci-haut)

0: "Coronavirus Brief" is a ORG found at https://en.wikipedia.org/wiki/Coronavirus_Brief
1: "CBC" is a ORG found at https://en.wikipedia.org/wiki/CBC
2: "Canada" is a GPE found at https://en.wikipedia.org/wiki/Canada
3: "C.D. Howe" is a PERSON found at https://en.wikipedia.org/wiki/C.D._Howe
4: "Monday" is a DATE found at https://en.wikipedia.org/wiki/Monday
5: "Alberta" is a GPE found at https://en.wikipedia.org/wiki/Alberta
6: "first" is a ORDINAL found at https://en.wikipedia.org/wiki/first
7: "Saturday" is a DATE found at https://en.wikipedia.org/wiki/Saturday
8: "Air Canada" is a ORG found at https://en.wikipedia.org/wiki/Air_Canada
9: "Christmas" is a DATE found at https://en.wikipedia.org/wiki/Christmas
10: "Canadians" is a NORP found at https://en.wikipedia.org/wiki/Canadians

12: "England" is a GPE found at https://en.wikipedia.org/wiki/England
13: "Peter Cziborra/Reuters" is a PERSON found at https://en.wikipedia.org/wiki/Peter_Cziborra/Reuters
14: "months" is a DATE found at

**(TO DO) Q7 - 5 points**  

Effectuez une évaluation qualitative de la méthode de linking d'entités que vous avez écrite en Q6. Pour votre évaluation qualitative, vous devez choisir un document (celui que vous voulez dans le corpus de nouvelles sur Covid-19, et assurez-vous de mentionner lequel) et exécuter votre méthode sur ce document. Répondez aux questions suivantes :

* a. Donnez 2 exemples d'entités où la forme plus longue a été trouvée dans Wikipedia. La page trouvée est-elle appropriée ? La forme plus courte serait-elle également trouvée? Serait-elle liée à la même page?
* b. Donnez 2 exemples d'entités où la page wikipedia n'existait pas. Pourquoi? La forme recherchée était-elle incorrecte ou l'entité était-elle peu connue (et donc sans page Wikipedia)?
* c. Essayez de restreindre votre recherche avec différents types d'entités. Pensez-vous que les DATE sont couvertes par Wikipédia ? Qu'en est-il de PERSON ou de GPE ? Discutez de la couverture des différents types d'entités en donnant des exemples.

**RÉPONSE Q7**
On va faire  une evaluation qualitative avec le document d'index 6

a-
2 exemples de d'entités où la forme plus longue a été trouvée dans Wikipedia:

38: "Calgary" is a GPE found at https://en.wikipedia.org/wiki/the_University_of_Calgary

36: "Habib" is a PERSON found at https://en.wikipedia.org/wiki/Ahsan_Habib

oui, La page trouvée est appropriée. 
La forme plus courte serait également trouvée mais ca ne serait pas liée à la même page

b-
2 exemples d'entités où la page wikipedia n'existait pas:

138: "Stead" is a ORG found at https://en.wikipedia.org/wiki/MJ_Stead

133: "Nancy Horsfall Couldwell" is a PERSON found at https://en.wikipedia.org/wiki/Nancy_Horsfall_Couldwell

Parce que Wikipedia n'a pas d'articles avec exactement ces entités. 
L'entité recherchée est surement peu connue et donc sans page Wikipedia.

c-

Oui, Les DATE sont couvertes par Wikipédia, de meme pour PERSON et  GPE

On trouve que les GPE sont beaucoup plus couverts par wikipedia que les autres types d'entités
surement parce qu'ils sont populaires, car certaines entités de type DATE ou PERSON sont peu connus et ne sont donc pas couverts par wikipedia. 
Ceci peut aussi s'expliquer par le fait que la forme normalisee de la plupart des GPE sont leur forme courte et donc plus de chances d'etre trouvées

par exemple:

42: "Quebec" is a GPE found at https://en.wikipedia.org/wiki/Quebec

5: "Alberta" is a GPE found at https://en.wikipedia.org/wiki/Alberta

27: "Berlin" is a GPE found at https://en.wikipedia.org/wiki/Berlin

28: "Budapest" is a GPE found at https://en.wikipedia.org/wiki/Budapest

29: "Mexico City" is a GPE found at https://en.wikipedia.org/wiki/Mexico_City


On note aussi quelques ambiguités, et ca fait que wikipedia ne sait pas quelle page est ce qu'on veut acceder 
par exemple:

20: "John Brown" is a PERSON found at https://en.wikipedia.org/wiki/John_Brown

qui donnent le resultat de plusieurs john brown

***SIGNATURE:***
Mon nom est Cheikh Bassirou Ndoye.
Mon numéro d'étudiant(e) est 300045791.
J'atteste être l'auteur(e) de ce Notebook.